<a href="https://colab.research.google.com/github/rkwi/master-thesis/blob/master/code/projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time
import numpy as np
import pandas as pd
from fenics import *
import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

In [0]:
def errornorm_L1_vector(u_e, u):
  V = u.function_space()
  mesh = V.mesh()
  degree = V.ufl_element().degree()
  W = VectorFunctionSpace(mesh, 'P', degree + 4)
  u_e_W = interpolate(u_e, W)
  u_W = interpolate(u, W)
  e_W = Function(W)
  e_W.vector()[:] = u_e_W.vector().get_local() - u_W.vector().get_local()
  error = (abs(e_W.sub(0))+abs(e_W.sub(1)))*dx
  return abs(assemble(error))

def errornorm_L1(u_e, u):
  V = u.function_space()
  mesh = V.mesh()
  degree = V.ufl_element().degree()
  W = FunctionSpace(mesh, 'P', degree + 4)
  u_e_W = interpolate(u_e, W)
  u_W = interpolate(u, W)
  e_W = Function(W)
  e_W.vector()[:] = u_e_W.vector().get_local() - u_W.vector().get_local()
  error = abs(e_W)*dx
  return abs(assemble(error))

def errornorm_Linf_vector(u_e, u):
  S = VectorFunctionSpace(u.function_space().mesh(),'P',u.function_space().ufl_element().degree() + 4)
  u_e_interpolated = interpolate(u_e,S)
  u_interpolated = interpolate(u,S)
  temp = u_e_interpolated.vector() - u_interpolated.vector()
  return norm(temp,'linf')

def errornorm_Linf(u_e, u):
  S = FunctionSpace(u.function_space().mesh(),'P',u.function_space().ufl_element().degree() + 4)
  u_e_interpolated = interpolate(u_e,S)
  u_interpolated = interpolate(u,S)
  temp = u_e_interpolated.vector() - u_interpolated.vector()
  return norm(temp,'linf')

def create_error_table(N_list, L1_error, L2_error, Linf_error):
  row = len(L1_error)
  data_array = np.zeros((row,6))
  for i in range(row):
    for j in range(6):
      if j == 0:
        data_array[i][j] = L1_error[i]
      elif j == 1 and i < row - 1:
        data_array[i][j] = np.log2(L1_error[i]/L1_error[i+1])
      elif j == 2:
        data_array[i][j] = L2_error[i]
      elif j == 3 and i < row - 1:
        data_array[i][j] = np.log2(L2_error[i]/L2_error[i+1])
      elif j == 4:
        data_array[i][j] = Linf_error[i]
      elif j == 5 and i < row - 1:
        data_array[i][j] = np.log2(Linf_error[i]/Linf_error[i+1])
  table = pd.DataFrame(data_array)
  table.set_index([[str('1/' + str(N)) for N in N_list]],inplace=True)
  table.columns = [['L1','L1_rate','L2','L2_rate','Linf','Linf_rate']]
  return table

In [0]:
N_list = [16,32,64,128]
u_list = []
u_exact_list = []
p_list = []
p_exact_list = []

for N in N_list:
  T = 0.5
  dt = 1.0/(2*N)
  k = Constant(dt)
  num_steps = int(T/dt)
  interpolation_degree = 3
  R = 1
  

  f = Expression(('(2*sin(t)*pow(sin(pi*x[0]), 2)*sin(pi*x[1]) + pi*sin(t)*sin(pi*x[0]) - 16*pow(pi, 2)*pow(sin(pi*x[0]), 2)*sin(pi*x[1])*cos(t) - 2*pow(pi, 3)*sin(pi*x[0])*cos(t) + 4*pow(pi, 2)*sin(pi*x[1])*cos(t))*cos(pi*x[1])'
                  ,'(-2*sin(t)*sin(pi*x[0])*pow(sin(pi*x[1]), 2) + pi*sin(t)*sin(pi*x[1]) + 16*pow(pi, 2)*sin(pi*x[0])*pow(sin(pi*x[1]), 2)*cos(t) - 4*pow(pi, 2)*sin(pi*x[0])*cos(t) - 2*pow(pi, 3)*sin(pi*x[1])*cos(t))*cos(pi*x[0])')
                 ,degree=interpolation_degree,t=0)
  
  f0 = Expression(('(2*sin(t)*pow(sin(pi*x[0]), 2)*sin(pi*x[1]) + pi*sin(t)*sin(pi*x[0]) - 16*pow(pi, 2)*pow(sin(pi*x[0]), 2)*sin(pi*x[1])*cos(t) - 2*pow(pi, 3)*sin(pi*x[0])*cos(t) + 4*pow(pi, 2)*sin(pi*x[1])*cos(t))*cos(pi*x[1])'
                  ,'(-2*sin(t)*sin(pi*x[0])*pow(sin(pi*x[1]), 2) + pi*sin(t)*sin(pi*x[1]) + 16*pow(pi, 2)*sin(pi*x[0])*pow(sin(pi*x[1]), 2)*cos(t) - 4*pow(pi, 2)*sin(pi*x[0])*cos(t) - 2*pow(pi, 3)*sin(pi*x[1])*cos(t))*cos(pi*x[0])')
                 ,degree=interpolation_degree,t=0)
  
  u_exact = Expression(('-pow(sin(pi*x[0]), 2)*sin(2*pi*x[1])*cos(t)'
                        ,'sin(2*pi*x[0])*pow(sin(pi*x[1]), 2)*cos(t)')
                       ,degree=interpolation_degree,t=0)

  p_exact = Expression('(-sin(t) + 2*pow(pi, 2)*cos(t))*cos(pi*x[0])*cos(pi*x[1])',degree=interpolation_degree,t=0)

  mesh = UnitSquareMesh(N,N)
  V = VectorFunctionSpace(mesh, 'P', 1)
  Q = FunctionSpace(mesh, 'P', 1)

  u = TrialFunction(V)
  phi = TrialFunction(Q)
  p = TrialFunction(Q)
  v = TestFunction(V)
  q = TestFunction(Q)

  u0 = Function(V)
  u1 = Function(V)
  u2 = Function(V)
  phi1 = Function(Q)
  p0 = Function(Q)
  p1 = Function(Q)
  
  def boundary_function(x, on_boundary):
    return on_boundary
  
  center = 'near(x[0],0.5) && near(x[1], 0.5)'
  bcu = [DirichletBC(V, (0,0), boundary_function)]
  bcphi = [DirichletBC(Q, 0, center, 'pointwise')]

  F1 = (1.0/k)*inner(u-u0,v)*dx + 0.5*(1/R)*inner(grad(u) + grad(u0),grad(v))*dx-0.5*inner(f + f0,v)*dx + inner(grad(p0),v)*dx
  a1 = lhs(F1)
  L1 = rhs(F1)

  a2 = inner(grad(phi),grad(q))*dx
  L2 = -(1.0/k)*div(u1)*q*dx

  a3 = p*q*dx
  L3 = (p0 + phi1)*q*dx
  
  a4 = inner(u,v)*dx
  L4 = inner(u1,v)*dx - k*inner(grad(p1) - grad(p0),v)*dx

  A1 = assemble(a1)
  A2 = assemble(a2)
  A3 = assemble(a3)
  A4 = assemble(a4)

  t = 0
  t_half = -0.5*dt
  for n in range(num_steps):

    t += dt
    t_half += dt
    f.t = t
    f0.t = t - dt
    u_exact.t = t
    p_exact.t = t_half

    b1 = assemble(L1)
    [bc.apply(A1, b1) for bc in bcu]
    solve(A1, u1.vector(), b1)

    b2 = assemble(L2)
    [bc.apply(A2, b2) for bc in bcphi]
    solve(A2, phi1.vector(), b2)

    b3 = assemble(L3)
    solve(A3, p1.vector(), b3)
    
    b4 = assemble(L4)
    [bc.apply(A4, b4) for bc in bcu]
    solve(A4, u2.vector(), b4)

    u0.assign(u2)
    p0.assign(p1)
    
  print('N =',N)

  u_list.append(u2)
  u_exact_list.append(u_exact)
  p_list.append(p1)
  p_exact_list.append(p_exact)
  
u_error_L2_list = []
u_error_L1_list = []
p_error_L2_list = []
p_error_L1_list = []
u_error_Linf_list = []
p_error_Linf_list = []

for i in range(len(u_list)):
  
  u_error_L2_list.append(errornorm(u_exact_list[i],u_list[i],'L2'))
  p_error_L2_list.append(errornorm(p_exact_list[i],p_list[i],'L2'))
  
  u_error_Linf_list.append(errornorm_Linf_vector(u_exact_list[i],u_list[i]))
  p_error_Linf_list.append(errornorm_Linf(p_exact_list[i],p_list[i]))
  
  u_error_L1_list.append(errornorm_L1_vector(u_exact_list[i],u_list[i]))
  p_error_L1_list.append(errornorm_L1(p_exact_list[i],p_list[i]))

In [0]:
u_table = create_error_table(N_list,u_error_L1_list,u_error_L2_list,u_error_Linf_list)

In [8]:
u_table

,L1,L1_rate,L2,L2_rate,Linf,Linf_rate
1/16,0.020069,1.848501,0.018587,1.809020,0.045169,1.312012
1/32,0.005573,1.912950,0.005304,1.744514,0.018192,0.967281
1/64,0.001480,1.961392,0.001583,1.671307,0.009305,0.950099
1/128,0.000380,0.000000,0.000497,0.000000,0.004816,0.000000


In [0]:
p_table = create_error_table(N_list,p_error_L1_list,p_error_L2_list,p_error_Linf_list)

In [10]:
p_table

,L1,L1_rate,L2,L2_rate,Linf,Linf_rate
1/16,0.144096,1.857859,0.185505,1.812705,0.628852,1.512890
1/32,0.039754,1.896739,0.052805,1.871165,0.220355,1.614893
1/64,0.010676,1.864210,0.014434,1.830157,0.071944,1.338182
1/128,0.002932,0.000000,0.004059,0.000000,0.028455,0.000000
